In [1]:
from games.tic_tac_toe import TicTacToeOpSp as TicTacToe
from src.game.agents import AIAgentOpSp as AIAgent, User
from algorithms.minimax_openspiel_wrapper import MiniMax
from explainers.alphabeta_explainer import AlphaBetaExplainer
from games.tic_tac_toe.interface.gradio_interface import TicTacToeGradioInterface

In [2]:
# Set the game state translator for consistent representations
from algorithms.minimax_openspiel_wrapper import TreeNode
TreeNode.game_state_translator = lambda cls, opsp_state: TicTacToe.opsp_state_to_action_space(opsp_state)

In [3]:
interface_mode = 'jupyter'

In [4]:
explainer = AlphaBetaExplainer()

opponent = AIAgent(agent_id=1, core=MiniMax(max_depth=30))
game = TicTacToe(players=[opponent, User(agent_id=0)],
                interface_mode=interface_mode, 
                interface_hyperlink_mode=interface_mode == 'gradio')

if interface_mode == 'gradio': # We need to start the interface externally
    game.explaining_agent = opponent
    interface = TicTacToeGradioInterface(game=game, explainer=explainer)

Simulate few initial moves:

In [5]:
simulate_initial_moves = False
if simulate_initial_moves:
    game.act({'where' : (0,0), 'who': 0})
    game.act({'where' : (1,1), 'who': 1})
    game.act({'where' : (1,0), 'who': 0})


# Play and Explain

In [6]:
if interface_mode == 'jupyter':
    await game.start_game()
else:
    interface.start()

In [10]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    explanation = explainer.explain(opponent.choice, 'the best') # can also modify explanation_depth=4, or print_depth=True
    print(explanation)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' 'X']] is the context
and 
[[' ' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' 'X']] o(1,1), id=0_4 is the best for me (in this context) (because
 
	[[' ' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' 'X']] o(1,1), id=0_4 has possible alternative moves 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] o(0,0), id=0_0, 
	[[' ' 'O' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] o(0,1), id=0_1, 
	[[' ' ' ' 'O']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] o(0,2), id=0_2, 
	[[' ' ' ' ' ']
	 ['O' ' ' ' ']
	 [' ' ' ' 'X']] o(1,0), id=0_3, 
	[[' ' ' ' ' ']
	 [' ' ' ' 'O']
	 [' ' ' ' 'X']] o(1,2), id=0_5, 
	[[' ' ' ' ' ']
	 [' ' ' ' ' ']
	 ['O' ' ' 'X']] o(2,0), id=0_6, 
	[[' ' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'O' 'X']] o(2,1), id=0_7
	and 
	[[' ' ' ' ' ']
	 [' ' 'O' ' ']
	 [' ' ' ' 'X']] o(1,1), id=0_4 is better for me than 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] o(0,0), id=0_0, 
	[[' ' 'O' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] o(0,1), id=0_1, 
	[[' ' ' ' 'O']
	 [' ' ' ' ' ']
	 [' ' ' ' '

In [8]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    explainer.frameworks['highlevel'].get_adjective('score').skip_statement = False
    explanation = explainer.explain(opponent.choice, 'the best') # can also modify explanation_depth=4, or print_depth=True
    print(explanation)

Print tree for debugging reasons:

In [9]:
if game.interface_mode == 'jupyter' and opponent.choice is not None:
    opponent.core.print_tree()